# Individual notebook: Giovanni Facchinetti (18202941)

## Introduction
In this individual notebook, we focus on the Python **dictionary** concrete data structure as an implementation of the **Hash Table** ADT.<br/><br/>
In our application, the very first step consists of reading, parsing and storing external data (in the form of csv files) in some kind of data structures that can be efficiently accessed and edited throughout the program.
- **Correctness**: ensured by the Python hash function that maps each key to an address number in the memory, assuring that a value can be accessed by a unique key
- **Speed** and **Efficiency**: Python dictionaries are fairly quick when accessing values given a key, as well as editing and deleting - normally O(1) time.
- **Robustness**: Python dictionaries can be considered robust enough. There are some restrictions that apply both to keys and values.
    - **Key restrictions**:
        - Keys must be unique, i.e. in the same dictionary they appear **only once**. Whenever we try to add a value with a key which is already present, Python overwrites the previous value with the new one - preserving the key.
        - keys must be of a type that is **hashable**, so that neither a list or another dictionary can act as keys in a dictionary since they are mutable data types. Hashable means that an object can be passed to the hash function which takes arbitrary size data and maps it to a fixed-size hash value used for table lookup and comparison. If we try to add a non-hashable key, Python throws a error.
    - **Value restrictions**:
        there are no restrictions on dictionary values. Any object type supported can be stored as a value, including user-defined objects. A value can also appear in a dictionary multiple times.
- **Clarity**: keys give some structure to the data and make the whole data body more human-readable and retrieavable.
- **Maintainability**: since the hash function maps every key to a unique value, it is fairly easy to maintain a dictionary. The syntax to change a value given a certain key is identical to the one used for lists (using indices).

## Design
We believe a Python **dictionary** data structure is best for our application, since its implementation of a symbol table provides a quick and efficient data retrieval by means of the built-in hash function.<br>

**N.B.**: what Python implements is not a perfect "hashing", but what it is known as "pre-hashing".
This means that Python has a mechanism to get a hash code from a key object, but it does not guarantee that the keyspace (the space containing all the possible keys) is restricted to a reasonable proportion of the table size.<br/>
Python "prehashes" an object to a non-negative integer using a built-in __hash__ function, which by default maps it to its memory address number.<br/>
Ideally, we would want hash(a) = hash(b) only if a = b; in Python it normally works except for some (rare case) hash(‘\0B ’) = 64 = hash(‘\0\0C ’).

Python dictionaries are well scalable, as the Python hash function is fairly robust. <br/>
By using a dictionary for the airport atlas, for instance, we ensure that every airport code is unique (acting as a key) and cannot be replicated in any way.
The access time is very small and in this notebook we will compare it with the Python **dynamic array** implementation, i.e. the **list** data structure. The main strength of a Python dictionary is the rapid access of stored values by means of a key, instead of an numeric index as it happens with arrays.

## Code

### Import packages
First of all, we import the modules we need to set-up our application.

In [32]:
import numpy as np
import pandas as pd

### Read data from provided datasets
We fetch the data from the provided csv files:
- __currencyrates.csv__ for currency rates in each country
- __countrycurrency.csv__ for country information and correspondent currencies used
- __airport.csv__ for airport information
- __aircraft.csv__ for aircraft models
- __test.csv__ for test cases

We import only the fields that are needed for our application and for each csv file we generate a dataframe object.

In [33]:
# Read currency data and extract needed fields
fields = ['CurrencyCode', 'toEuro']
df = pd.read_csv("currencyrates.csv", skipinitialspace = True, usecols = fields)

# Read country data and extract needed fields
fields = ['name','currency_alphabetic_code']
df_2 = pd.read_csv("countrycurrency.csv", skipinitialspace=True, usecols=fields)

# Read airport data and extract needed fields
# In this case we need to use column indices as the data lacks headers
# After having examined the csv file with a spreadsheet editor, we rename the columns we need
df_3 = pd.read_csv("airport.csv", header = None, usecols = [3,4,6,7])
df_3.columns = ['Country_name', 'Airport_Code', 'Latitude', 'Longitude']

# Read aircraft data and extract needed fields
fields = ['code', 'range']
df_4 = pd.read_csv("aircraft.csv", skipinitialspace = True, usecols = fields)

# Read test cases from csv file
df_5 = pd.read_csv("test.csv", header=None)

### Merge fetched data in a new dataframe
After having successfully imported external data, we proceed to merge the separated dataframes in a new dataframe object that we will work on.

In [34]:
# Merge currency rates and country dataframes
merged_currency_country_exchange = pd.merge(df, df_2, how = 'inner', left_on = ['CurrencyCode'], right_on = ['currency_alphabetic_code'])

In [35]:
# Merge the newly created dataframe with the airport information
merged_currency_country_exchange_airport_info_lat_long = pd.merge(merged_currency_country_exchange, df_3, how = 'outer', left_on = ['name'], right_on = ['Country_name'])

### Examine the new dataframe

In [36]:
print("The dataset has %s rows and %s columns." % merged_currency_country_exchange_airport_info_lat_long.shape)

The dataset has 5855 rows and 8 columns.


In [37]:
# Show dataframe first 50 rows
merged_currency_country_exchange_airport_info_lat_long.head(50)

,CurrencyCode,toEuro,name,currency_alphabetic_code,Country_name,Airport_Code,Latitude,Longitude
0,AED,0.258400,United Arab Emirates,AED,United Arab Emirates,AUH,24.432972,54.651138
1,AED,0.258400,United Arab Emirates,AED,United Arab Emirates,AZI,24.428333,54.458084
2,AED,0.258400,United Arab Emirates,AED,United Arab Emirates,DXB,25.252778,55.364444
3,AED,0.258400,United Arab Emirates,AED,United Arab Emirates,FJR,25.112225,56.323964
4,AED,0.258400,United Arab Emirates,AED,United Arab Emirates,RKT,25.613483,55.938817
5,AED,0.258400,United Arab Emirates,AED,United Arab Emirates,SHJ,25.328575,55.517150
6,AED,0.258400,United Arab Emirates,AED,United Arab Emirates,AAN,24.261667,55.609167
7,AED,0.258400,United Arab Emirates,AED,United Arab Emirates,NHD,25.026940,55.366110
8,AED,0.258400,United Arab Emirates,AED,United Arab Emirates,DWC,24.550560,55.103174
9,AED,0.258400,United Arab Emirates,AED,United Arab Emirates,XSB,24.285608,52.578347


### Drop unwanted / unneccessary values

In [38]:
# Drop 'na' values
merged_currency_country_exchange_airport_info_lat_long.dropna(how = 'any',inplace = True) 

### Drop redundant columns
After merging, we have redundant columns in our dataframe that should be dropped.

In [39]:
# Drop currency alphabetic code
merged_currency_country_exchange_airport_info_lat_long = merged_currency_country_exchange_airport_info_lat_long.drop('currency_alphabetic_code', 1)

In [40]:
# Drop country name
merged_currency_country_exchange_airport_info_lat_long = merged_currency_country_exchange_airport_info_lat_long.drop('name', 1)

In [41]:
print("The dataset has now %s rows and %s columns." % merged_currency_country_exchange_airport_info_lat_long.shape)

The dataset has now 5832 rows and 6 columns.


## Generate and populate the data structure
Now that we fetched our data in Pandas dataframe objects, we can now store them in the two data structures compared in this Notebook, testing the time they take to organize the same data.

We will populate the two data structures with the following data fields:
- **airport IATA code**
- **airport latitude**
- **airport longitude**
- **country currency code**
- **country exchange rate**
- **country extended name**

We will also test the amount of time required by the interpreter to generate and populate the data structures.<br/><br/>
**N.B.**: to populate our airport atlas data structures, we need to iterate over all the rows in our dataframe - by means of a **single for loop** -  so that the upper-bound time complexity is **O(n)**.

### Python list
We store our airport atlas using a list: all data fields will be list items accessible by indices.<br/>
Since Python treats lists as dynamic arrays, it allocates memory dynamically as the list size grows - capacity is always greater than equal than the list size.
We bypass this "pythonic" behaviour by **limiting the list to a fixed size**, given by the amount of rows in the airport dataframe.
This way, we ensure list are treated as **classic fixed-size arrays**.<br/>

What we obtain is thus a multidimensional list, or a "list of lists", where the rows are represented by the n-outer list elements consisting of m items each, with m = number of columns in our dataframe.

In [195]:
%%time

# First element in shape tuple is the number of rows
count_row = merged_currency_country_exchange_airport_info_lat_long.shape[0]

# Ensure better memory consumption
# Size already pre-defined so that there is no uneccesary waste of memory
list_alternative_to_dictionary = [0] * count_row

# Add data items iterating over dataframe rows
i = 0
for index, row in merged_currency_country_exchange_airport_info_lat_long.iterrows():
    list_alternative_to_dictionary[i] = [row['Airport_Code'], row['Latitude'], row['Longitude'], row['CurrencyCode'], row['toEuro'], row['Country_name']]
    i += 1
    

#print(list_alternative_to_dictionary)

CPU times: user 561 ms, sys: 19.2 ms, total: 580 ms
Wall time: 593 ms


### Python dictionary
In order to populate our new dictionary, we will use the airport IATA code as the unique key. It will act as a reference every time we will need to search for an airport information.
Our solution is in some way an hybrid between arrays and hash tables, as the dictionary values will be lists, with each list item representing all the other information we need.

In [304]:
%%time

# Initialize empty dictionary
dictionary = {}

# Add data items iterating over dataframe rows
for index, row in merged_currency_country_exchange_airport_info_lat_long.iterrows():
     dictionary[row['Airport_Code']] = [row['Latitude'], row['Longitude'],row['CurrencyCode'],row['toEuro'],row['Country_name']]
#print(dictionary)        

CPU times: user 786 ms, sys: 151 ms, total: 937 ms
Wall time: 1.07 s


In order to gain a comprehensive view of our dictionary, we run the following command which output shows all the keys stored in it:

In [137]:
# Show dict keys
print(dictionary.keys())

dict_keys(['AUH', 'AZI', 'DXB', 'FJR', 'RKT', 'SHJ', 'AAN', 'NHD', 'DWC', 'XSB', 'ZDY', 'HEA', 'JAA', 'KBL', 'KDH', 'MMZ', 'MZR', 'UND', 'FBD', 'BPM', 'TII', 'ZAJ', 'CCN', 'KHT', 'AZ3', 'BST', 'BIN', 'TIA', 'EVN', 'LWN', 'CUR', 'SXM', 'SSY', 'BUG', 'CAB', 'NOV', 'SVP', 'LAD', 'MEG', 'SPP', 'GXG', 'PBN', 'VHC', 'SZA', 'SDD', 'LUO', 'UGO', 'XGN', 'MSZ', 'VPE', 'DUE', 'CBT', 'LBZ', 'KNP', 'RAF', 'COC', 'GHU', 'PRA', 'ROS', 'SFN', 'AEP', 'COR', 'LPG', 'MDZ', 'LGS', 'AFA', 'CTC', 'SDE', 'IRJ', 'TUC', 'UAQ', 'RCU', 'VDR', 'LUQ', 'CNQ', 'RES', 'FMA', 'IGR', 'AOL', 'PSS', 'SLA', 'JUJ', 'ORA', 'EHL', 'CRD', 'EQS', 'REL', 'VDM', 'PMY', 'PUD', 'RGA', 'RGL', 'USH', 'ULA', 'PMQ', 'RZA', 'BHI', 'MDQ', 'NQN', 'RSA', 'BRC', 'TDL', 'VLG', 'CPC', 'EZE', 'FTE', 'SST', 'GGS', 'OES', 'LHS', 'TTG', 'NEC', 'ING', 'APZ', 'RDS', 'OLN', 'RYO', 'CVI', 'SGV', 'IGB', 'ELX', 'RLO', 'ARR', 'JSM', 'CFX', 'RHD', 'MTL', 'DGE', 'WSY', 'NSO', 'CES', 'OKB', 'ABM', 'ASP', 'BNE', 'OOL', 'CNS', 'CTL', 'ISA', 'MCY', 'MKY', 'P

### Considerations
As we can note from the two tests above, creating (and populating) a dictionary requires slightly more time than the same activity involving a fixed-size list.

## Time testing
In order to test between the two ADT Python implementations, we can measure the time required to 3 basic operations that may be carried out:
- **access**
- **insert**
- **delete**

### Access

#### List

For instance, we want to access information about a single airport in the list with index 5000:

In [289]:
%%timeit -n1
# Accessing a list
list_alternative_to_dictionary[5000]

The slowest run took 52.28 times longer than the fastest. This could mean that an intermediate result is being cached.
1.19 µs ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Dictionary

For the same purpose, we want to access information about a single airport in the dictionary - we search for Amsterdam Schipol's information through its unique key 'AMS':

In [290]:
%%timeit -n1
# Accessing a dictionary
dictionary['AMS']

The slowest run took 269.59 times longer than the fastest. This could mean that an intermediate result is being cached.
6.73 µs ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [291]:
%%timeit -n1
# Alternative way to access a value from a key
dictionary.get('AMS')

The slowest run took 7.36 times longer than the fastest. This could mean that an intermediate result is being cached.
651 ns ± 609 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Insert

We try to insert in our data structures a mock example of an airport.
First, we create a copy of our dictionary and list using the Python **copy()** method:

**N.B.**: we avoid **timeit** as we want to insert and delete only once. Instead, we use **time**.

In [292]:
# Make a copy of the two ds
dcopy = dictionary.copy()
lcopy = list_alternative_to_dictionary.copy()

#### List

In [293]:
%%time
lcopy.insert(5000, ['FSL', 100, 50, 'FSL', 129, 'Fantasyland'])

CPU times: user 29 µs, sys: 14 µs, total: 43 µs
Wall time: 49.1 µs


In [295]:
# Check if successfully inserted
lcopy[5000]

['FSL', 100, 50, 'FSL', 129, 'Fantasyland']

#### Dictionary

In [297]:
%%time
dcopy['FSL'] = [100, 50, 'FSL', 129, 'Fantasyland']

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.06 µs


In [298]:
# Check if successfully inserted
dcopy.get('FSL')

[100, 50, 'FSL', 129, 'Fantasyland']

### Delete

#### List

In [299]:
%%time
del lcopy[5000]

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


In [300]:
# Check if successfully deleted
lcopy[5000]

['KLW', 55.579167000000005, -133.076111, 'USD', 0.9488, 'United States']

#### Dictionary

In [301]:
%%time
del dcopy['FSL']

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


In [302]:
# If empty, element successfully deleted
dcopy.get('FSL')

### Considerations (2)
- **access** requires consistently more time for the dictionary data structure.
- **insert** requires consistently more time for the list data structure, making the dictionary more efficient to maintain.
- **delete** requires a comparable time on average for both the data structures examined.

## Unit test
In this section, we use the Unittest class to test the dictionary data structure.

In [107]:
import unittest

### Basic tests

In [194]:
class TestDict(unittest.TestCase):
    """Basic tests on the Python dictionary data structure"""
    
    d1 = {'a': 1, 'b': 2, 'c': [1, 2]}
    d2 = {'a': 1, 'b': 2, 'c': [1]}
    d3 = {'a': 1, 'b': 2, 'c': [1]}
    
    def test_key(self):
        self.assertEqual(hash('key'),hash('key'))

    def test_key2(self):
        self.assertFalse(hash('key') == hash(5))

    def test_edit(self):
        self.d1['c'] = [1]
        self.assertDictEqual(self.d1, self.d2)
        
    def test_add(self):
        self.d3['d'] = "mystring"
        self.assertEqual(self.d3.get('d'), "mystring")
        
    def test_remove(self):
        del self.d3['d']
        self.assertDictEqual(self.d3, {'a': 1, 'b': 2, 'c': [1]})
    
    unittest.main(argv=[''], verbosity = 2, exit = False)

test_add (__main__.TestDict) ... ok
test_edit (__main__.TestDict) ... ok
test_key (__main__.TestDict) ... ok
test_key2 (__main__.TestDict) ... ok
test_remove (__main__.TestDict) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.004s

OK


## Summary

From time complexity inspection we ascertain that the **creation of a dictionary** occurs in slightly more time than the **creation of the list**.<br/>
The same applies to **accessing of information by key in the dictionary** and **accessing an element of the list** at time of writing.<br/>
An added point in relation to the choice of using a dictionary as a data structure to store all the neccessary information from the data frame is the **accessibility of the value by the actual key**.<br>

There is a **trade off** between the loss of efficiency in terms of time required to execute basic operations with a dictionary when compared to a simple Python list (dynamic array, in this context we keep it fixed thus saving memory space). But still, we think that the dictionary data structure implementation of a hash table is preferred for its maintainability and for the ease of accessing items by means of a unique key (representing each airport, for instance).
<br>

The structure of the dictionary lends itself to ease of coding and debugging.<br>

The main reason why we chose to use a Python dictionary to store the airport atlas is to take advantage of its robustness and maintainability as a database, on which develop every algorithm needed to the most efficient route among all the possible routes.